## Exploratory Data Analysis on FHIR

In [1]:
#!pip install pandas
#!pip install requests
#!pip install altair 

NOTE: must be anonymized

### Install Anaconda environment

### Install Python SMART on FHIR client 

See docs: http://docs.smarthealthit.org/client-py/

### Generate test data

Download FHIR STU3 sample data from Synthea
https://synthetichealth.github.io/synthea/

In [3]:
import os

DATA_PATH = os.path.join(".", "data")
FHIR_1K_PATIENTS_PATH = os.path.join(DATA_PATH, "1k-patients")

!mkdir {DATA_PATH}
!ls
!mkdir {FHIR_1K_PATIENTS_PATH}
!ls {FHIR_1K_PATIENTS_PATH}

mkdir: ./data: File exists
FHIR.ipynb __MACOSX   data
mkdir: ./data/1k-patients: File exists
1k-patients.zip          all_prevalences.csv      stripped_prevalences.csv
__MACOSX                 fhir


In [5]:
import requests

SAMPLE_DATA_URL="https://syntheticmass.mitre.org/downloads/2017_11_06/synthea_sample_data_fhir_stu3_nov2017.zip"
response = requests.get(SAMPLE_DATA_URL)

DATA_PATH = os.path.join(".", "data")
FHIR_1K_PATIENTS_PATH = os.path.join(DATA_PATH, "1k-patients")
FHIR_1K_PATIENTS_ZIP_PATH = os.path.join(FHIR_1K_PATIENTS_PATH, "1k-patients.zip")

file = open(FHIR_1K_PATIENTS_ZIP_PATH, 'wb')
file.write(response.content)
file.close()

We got 20MB zip file

In [6]:
!ls -la {FHIR_1K_PATIENTS_PATH}

total 39040
drwxr-xr-x     7 antti  staff       224 Nov  9 14:09 .
drwxr-xr-x     3 antti  staff        96 Nov  9 12:36 ..
-rw-r--r--     1 antti  staff  19957256 Nov  9 16:00 1k-patients.zip
drwxr-xr-x     3 antti  staff        96 Nov  9 12:40 __MACOSX
-rw-r--r--     1 antti  staff     13916 Nov  9 12:40 all_prevalences.csv
drwxr-xr-x  1465 antti  staff     46880 Nov  9 12:40 fhir
-rw-r--r--     1 antti  staff      9368 Nov  9 14:09 stripped_prevalences.csv


In [41]:
import zipfile
zip = zipfile.ZipFile(FHIR_1K_PATIENTS_ZIP_PATH)
zip.extractall(FHIR_1K_PATIENTS_PATH)
zip.close()

In [8]:
!ls -ls {FHIR_1K_PATIENTS_PATH}

total 39040
38984 -rw-r--r--     1 antti  staff  19957256 Nov  9 16:00 1k-patients.zip
    0 drwxr-xr-x     3 antti  staff        96 Nov  9 12:40 __MACOSX
   32 -rw-r--r--     1 antti  staff     13916 Nov  9 16:00 all_prevalences.csv
    0 drwxr-xr-x  1465 antti  staff     46880 Nov  9 12:40 fhir
   24 -rw-r--r--     1 antti  staff      9368 Nov  9 14:09 stripped_prevalences.csv


In [9]:
!ls -lS {FHIR_1K_PATIENTS_PATH}/fhir | head -10

total 575808
-rw-r--r--  1 antti  staff  954679 Nov  9 16:00 Labadie956_Jeff209_19.json
-rw-r--r--  1 antti  staff  918365 Nov  9 16:00 Flatley187_Frank245_15.json
-rw-r--r--  1 antti  staff  864132 Nov  9 16:00 Volkman764_Caron629_51.json
-rw-r--r--  1 antti  staff  795994 Nov  9 16:00 Spencer562_Dallas785_55.json
-rw-r--r--  1 antti  staff  774497 Nov  9 16:00 hospital_information.json
-rw-r--r--  1 antti  staff  733798 Nov  9 16:00 Prosacco693_Annie110_23.json
-rw-r--r--  1 antti  staff  732881 Nov  9 16:00 McCullough769_Martha55_32.json
-rw-r--r--  1 antti  staff  724654 Nov  9 16:00 Kreiger821_Patty803_21.json
-rw-r--r--  1 antti  staff  676237 Nov  9 16:00 Cole124_Abbie484_21.json


Ok, so there is `all_prevalences.csv` file and `fhir` directory which contains the JSONs. The largest JSON files are ~1MB. And in addition to patient related JSON files, there is also other type of information, e.g. hospital information.

### What's in `all_prevalences.csv`?

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv(os.path.join(FHIR_1K_PATIENTS_PATH, "all_prevalences.csv"))
df.head()

,ITEM,POPULATION TYPE,OCCURRENCES,POPULATION COUNT,PREVALENCE RATE,PREVALENCE PERCENTAGE
0,Viral Sinusitis (Disorder),LIVING,868,1000,0.868,86.8
1,Streptococcal Sore Throat (Disorder),LIVING,487,1000,0.487,48.7
2,Laceration Of Thigh,LIVING,117,1000,0.117,11.7
3,Acute Viral Pharyngitis (Disorder),LIVING,772,1000,0.772,77.2
4,Acute Bronchitis (Disorder),LIVING,749,1000,0.749,74.9


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 6 columns):
ITEM                     244 non-null object
POPULATION TYPE          244 non-null object
OCCURRENCES              244 non-null int64
POPULATION COUNT         244 non-null int64
PREVALENCE RATE          244 non-null float64
PREVALENCE PERCENTAGE    244 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 11.5+ KB


In [13]:
df.describe()

,OCCURRENCES,POPULATION COUNT,PREVALENCE RATE,PREVALENCE PERCENTAGE
count,244.000000,244.0,244.000000,244.000000
mean,72.819672,1000.0,0.072820,7.281967
std,135.550638,0.0,0.135551,13.555064
min,0.000000,1000.0,0.000000,0.000000
25%,6.000000,1000.0,0.006000,0.600000
50%,25.000000,1000.0,0.025000,2.500000
75%,70.500000,1000.0,0.070500,7.050000
max,868.000000,1000.0,0.868000,86.800000


In [14]:
df["POPULATION TYPE"].value_counts()

LIVING    244
Name: POPULATION TYPE, dtype: int64

In [15]:
df["POPULATION COUNT"].value_counts()

1000    244
Name: POPULATION COUNT, dtype: int64

Let's just remove population type and count columns as they don't provide any information. 

In [16]:
# axis=1 to drop columns instead of rows
# inplace=True to modify the data frame inplace, returns None instead of the data frame
df.drop(["POPULATION TYPE", "POPULATION COUNT"], axis=1, inplace=True)
df.head()

,ITEM,OCCURRENCES,PREVALENCE RATE,PREVALENCE PERCENTAGE
0,Viral Sinusitis (Disorder),868,0.868,86.8
1,Streptococcal Sore Throat (Disorder),487,0.487,48.7
2,Laceration Of Thigh,117,0.117,11.7
3,Acute Viral Pharyngitis (Disorder),772,0.772,77.2
4,Acute Bronchitis (Disorder),749,0.749,74.9


Occurrences, prevalence rate and prevalence percentage all provide the same information. Let's keep just one of them and drop others.

In [17]:
df.drop(["PREVALENCE RATE", "PREVALENCE PERCENTAGE"], axis=1, inplace=True)
df.head()

,ITEM,OCCURRENCES
0,Viral Sinusitis (Disorder),868
1,Streptococcal Sore Throat (Disorder),487
2,Laceration Of Thigh,117
3,Acute Viral Pharyngitis (Disorder),772
4,Acute Bronchitis (Disorder),749


Items with highest and lowest prevalence rates:

In [18]:
df.nlargest(10, "OCCURRENCES")

,ITEM,OCCURRENCES
0,Viral Sinusitis (Disorder),868
3,Acute Viral Pharyngitis (Disorder),772
4,Acute Bronchitis (Disorder),749
8,Otitis Media,699
141,Acetaminophen 160 Mg,689
164,Acetaminophen 160 Mg Oral Tablet,536
135,Amoxicillin 250 Mg / Clavulanate 125 Mg [Augme...,534
1,Streptococcal Sore Throat (Disorder),487
136,Penicillin V Potassium 250 Mg,390
139,Acetaminophen 325 Mg Oral Tablet,388


In [19]:
df.nsmallest(10, "OCCURRENCES")

,ITEM,OCCURRENCES
13,Suspected Lung Cancer (Situation),0
14,Non Small Cell Lung Cancer (Disorder),0
15,Non Small Cell Carcinoma Of Lung Tnm Stage 4 (...,0
78,Pneumonia,0
81,Small Cell Carcinoma Of Lung (Disorder),0
82,Primary Small Cell Malignant Neoplasm Of Lung ...,0
104,History Of Amputation Of Foot (Situation),0
115,Non Small Cell Carcinoma Of Lung Tnm Stage 1 (...,0
126,Non Small Cell Carcinoma Of Lung Tnm Stage 3 (...,0
127,Burn Injury(Morphologic Abnormality),0


Items are disorders, situations, medication...

In [20]:
df.to_csv(os.path.join(FHIR_1K_PATIENTS_PATH, "stripped_prevalences.csv"))

In [21]:
!ls {FHIR_1K_PATIENTS_PATH}

1k-patients.zip          all_prevalences.csv      stripped_prevalences.csv
__MACOSX                 fhir


### Exploring JSONs

In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

Questions to ask?
Examples:

What types of cars are most likely to be pulled over for speeding?
What times of day are police most active?
How common are "speed traps"? Or are tickets spread pretty evenly in terms of geography?
What are the most common things people are pulled over for?

Let's see what kind of structure a FHIR JSON has. Let's first find the smallest JSON.

In [23]:
JSONS_PATH = os.path.join(FHIR_1K_PATIENTS_PATH, "fhir")
!ls -lSr {JSONS_PATH} | head -4

total 575808
-rw-r--r--  1 antti  staff    8858 Nov  9 16:00 Pfannerstill249_Richelle643_0.json
-rw-r--r--  1 antti  staff   13835 Nov  9 16:00 Hamill916_Donette817_0.json
-rw-r--r--  1 antti  staff   15134 Nov  9 16:00 Herzog829_Delana160_0.json


In [40]:
JSON_PATH = os.path.join(JSONS_PATH, "Pfannerstill249_Richelle643_0.json")
!head {JSON_PATH}

https://www.dataquest.io/blog/python-json-tutorial/
"From this, we can tell that the JSON data is a dictionary, and is well formatted. rypw is a top level key, and is indented two spaces."

In [25]:
import json
from pandas.io.json import json_normalize

In [44]:
with open(JSON_PATH) as json_data:
    d = json.load(json_data)

In [45]:
type(d)

dict

In [50]:
d.keys()

dict_keys(['type', 'entry', 'resourceType'])

In [59]:
d['type']

'collection'

collection	Collection	The bundle is a set of resources collected into a single package for ease of distribution.
https://www.hl7.org/fhir/valueset-bundle-type.html

In [60]:
d['resourceType']

'Bundle'

In [62]:
#d['entry']

In [64]:
bundle_df = json_normalize(d['entry'])
bundle_df.head()

,fullUrl,resource.address,resource.birthDate,resource.category,resource.code.coding,resource.code.text,resource.communication,resource.effectiveDateTime,resource.extension,resource.gender,...,resource.status,resource.subject.reference,resource.telecom,resource.text.div,resource.text.status,resource.type,resource.valueQuantity.code,resource.valueQuantity.system,resource.valueQuantity.unit,resource.valueQuantity.value
0,urn:uuid:94bec074-5c4b-440c-ba5e-409cd9a896a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[{'coding': [{'system': 'http://hl7.org/fhir/o...,NaN,NaN,NaN,NaN
1,urn:uuid:ecdaa313-efc9-4080-aee1-aa212271ddbf,[{'extension': [{'extension': [{'url': 'latitu...,2017-11-07,NaN,NaN,NaN,[{'language': {'coding': [{'system': 'http://h...,NaN,[{'url': 'http://hl7.org/fhir/us/core/Structur...,female,...,NaN,NaN,"[{'system': 'phone', 'value': '864-009-1257', ...","<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",generated,NaN,NaN,NaN,NaN,NaN
2,urn:uuid:f6176648-8ccd-4fd3-b67e-06de4a84fced,NaN,NaN,[{'coding': [{'system': 'http://hl7.org/fhir/o...,"[{'system': 'http://snomed.info/sct', 'code': ...",Quality adjusted life years,NaN,2017-10-31T16:09:28-04:00,NaN,NaN,...,final,urn:uuid:ecdaa313-efc9-4080-aee1-aa212271ddbf,NaN,NaN,NaN,NaN,years,http://unitsofmeasure.org/,years,0.0
3,urn:uuid:fae912c9-db09-4428-a769-3a4a18c2ef0e,NaN,NaN,[{'coding': [{'system': 'http://hl7.org/fhir/o...,"[{'system': 'http://snomed.info/sct', 'code': ...",Disability rating scale,NaN,2017-10-31T16:09:28-04:00,NaN,NaN,...,final,urn:uuid:ecdaa313-efc9-4080-aee1-aa212271ddbf,NaN,NaN,NaN,NaN,years,http://unitsofmeasure.org/,years,0.0


https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
"We see more nested columns: resource.category etc.. json_normalize docs give us some hints how to flatten semi-structured data further. Let's unpack the works column into a standalone dataframe. We'll also grab the flat columns so we can do analysis. "